## Script que gera os arquivos base para o aprendizado do Corruptometro.

In [52]:
import pandas as pd
import numpy as np
import unicodedata

def padronizaTexto(texto):
    """
    Remove acentos e coloca em caixa alta todo o texto que necessita ser padronizado.
    
    """
    return unicodedata.normalize('NFKD', texto.decode('utf-8')).encode('ASCII', 'ignore').upper()

### Manipulação das informações da UNIDADE GESTORA

In [62]:
ANO_GESTAO_2009, ANO_GESTAO_2012, ANO_GESTAO_2013, ANO_GESTAO_2016 = 2009, 2012, 2013, 2016

def mudaAnoUgestora(ano):
    """
    Troca o valor do dt_Ano pelo ano do inicio da gestão.
    Isso será necessário para unir com a tabela de eletos e a de inidoneidade no futuro.
    """
    inicio_gestao_2009, inicio_gestao_2013 = 2009, 2013
    if ano >= ANO_GESTAO_2009 and ano <= ANO_GESTAO_2012:
        return str(inicio_gestao_2009)
    elif ano >= ANO_GESTAO_2013 and ano <= ANO_GESTAO_2016:
        return str(inicio_gestao_2013)

def nomeUgestora(gestora):
    """
    Remove a frase 'Prefeitura Municipal de'todos os nomes de unidade gestora.

    """
    QUANT_CARACTER_REMOVE = 24
    return padronizaTexto(gestora[QUANT_CARACTER_REMOVE:])

def manipulaUnidadeGestora():
    """
    Retorna o código da unidade gestora, o ano do inicio da gestão e a unidade gestora para todas as prefeituras
    a partir de 2009.

    """    
    path_ges = r'~/Downloads/codigo_ugestora.csv'
    ud_gestoras = pd.read_csv(path_ges) 
    ud_gestoras_pref = ud_gestoras[(ud_gestoras['dt_Ano'] > 2008) & 
                                   (ud_gestoras['de_Ugestora'].str.contains("Prefeitura"))]
    func_anos, func_nome = np.vectorize(mudaAnoUgestora), np.vectorize(nomeUgestora)
    anos, u_gestoras = ud_gestoras_pref['dt_Ano'].values, ud_gestoras_pref['de_Ugestora'].values
    nov_anos, nov_u_gestoras = func_anos(anos), func_nome(u_gestoras)
    ud_gestoras_pref.loc[:,('dt_Ano')], ud_gestoras_pref.loc[:,('de_Ugestora')] = nov_anos, nov_u_gestoras
    return ud_gestoras_pref[['cd_Ugestora','dt_Ano','de_Ugestora']]

ud_gestoras_pref_pos_2008 = manipulaUnidadeGestora()
ud_gestoras_pref_pos_2008.head()

,cd_Ugestora,dt_Ano,de_Ugestora
1666,201001,2009,AGUA BRANCA
1667,201001,2009,AGUA BRANCA
1668,201001,2009,AGUA BRANCA
1669,201001,2009,AGUA BRANCA
1670,201001,2013,AGUA BRANCA


## Manipulação da base de dados de candidatos eleitos 2008-2012

In [58]:

def padronizaNomePrefeito(nome):
    """
    Retira acentos e padroniza o nome do prefeito para maiusculo.
    """
    return padronizaTexto(nome)


def anoAposEleicao(ano):
    """
    Para termos como referencia o ano em que o prefeito começou sua gestão. Não o ano em que ele foi eleito.

    """
    ELEICAO_2008, GESTAO_2009, ELEICAO_2012, GESTAO_2013 = '2008', '2009', '2012', '2013'
    if ano == ELEICAO_2008:
        return GESTAO_2009
    elif ano == ELEICAO_2012:
        return GESTAO_2013


def manipulaCandidatosEleitos():
    """
    Manipula os dados das eleicoes para selecionar apenas os prefeitos eleitos entre 2008 e 2012.
    Ajustando o ano em que começaram seu mandato.

    """
    COD_PREFEITO = '11' 
    path_cand_2008, path_cand_2012 = r'~/Downloads/candidate_local_2008-1.csv', r'~/Downloads/candidate_local_2012-1.csv'
    cand_2008, cand_2012 = pd.read_csv(path_cand_2008, dtype='unicode'), pd.read_csv(path_cand_2012, dtype='unicode')
    eleicoes2008_2012 = [cand_2008,cand_2012]
    cand_2008_2012 = pd.concat(eleicoes2008_2012)
    
    cand_2008_2012_PB_ELEITOS = cand_2008_2012[(cand_2008_2012['SIGLA_UF'] == 'PB') &
                                               (cand_2008_2012['DESC_SIT_TOT_TURNO'] == 'ELEITO') &
                                              (cand_2008_2012['CODIGO_CARGO'] == COD_PREFEITO)]
    
    func_ano_pos_eleicao, func_nome_prefeit = np.vectorize(anoAposEleicao), np.vectorize(padronizaNomePrefeito)
    anos_eleic, nomes_pref_eleit = cand_2008_2012_PB_ELEITOS['ANO_ELEICAO'].values, cand_2008_2012_PB_ELEITOS['NOME_CANDIDATO'].values
    anos_pos_eleicao, nomes_pref_eleitos = func_ano_pos_eleicao(anos_eleic), func_nome_prefeit(nomes_pref_eleit)
    cand_2008_2012_PB_ELEITOS.loc[:,('ANO_ELEICAO')] = anos_pos_eleicao
    cand_2008_2012_PB_ELEITOS.loc[:,('NOME_CANDIDATO')] = nomes_pref_eleitos
    
    cand_2008_2012_PB_ELEITOS.rename(columns={'DESCRICAO_UE': 'de_Ugestora', 'ANO_ELEICAO':'dt_Ano'}, inplace=True)
    return cand_2008_2012_PB_ELEITOS[['dt_Ano','de_Ugestora', 'NOME_CANDIDATO']]


cand_2008_2012_PB_ELEITOS = manipulaCandidatosEleitos()
cand_2008_2012_PB_ELEITOS.head()

,dt_Ano,de_Ugestora,NOME_CANDIDATO
184412,2009,AGUIAR,MANOEL BATISTA GUEDES FILHO
184438,2009,ALAGOA GRANDE,JOAO BOSCO CARNEIRO JUNIOR
184530,2009,ALAGOA NOVA,KLEBER HERCULANO DE MORAES
184581,2009,ALAGOINHA,ALCIONE MARACAJA DE MORAIS BELTRAO
184633,2009,ALCANTIL,JOSE MILTON RODRIGUES


## Amplia a tabela Sagres com as informacoes Eleitos por Ano.

In [64]:
sagres_pref_eleit_por_ano = pd.merge(ud_gestoras_pref_pos_2008,cand_2008_2012_PB_ELEITOS,on=['de_Ugestora', 'dt_Ano'])
sagres_pref_eleit_por_ano = sagres_pref_eleit_por_ano.drop_duplicates()
sagres_pref_eleit_por_ano.head()

,cd_Ugestora,dt_Ano,de_Ugestora,NOME_CANDIDATO
0,201002,2009,AGUIAR,MANOEL BATISTA GUEDES FILHO
4,201002,2013,AGUIAR,MANOEL BATISTA GUEDES FILHO
8,201003,2009,ALAGOA GRANDE,JOAO BOSCO CARNEIRO JUNIOR
12,201003,2013,ALAGOA GRANDE,HILDON REGIS NAVARRO FILHO
16,201004,2009,ALAGOA NOVA,KLEBER HERCULANO DE MORAES


## Manipula a base de gestores condenados pelo TRE.

In [78]:
def anoPublPGestao(ano):
    """
    Transforma o ano da data de publicação em ano do início de mandato.
    
    """
    #Acrescenta 20 ao final da data 12/03/09 - 20+09 = 2009
    ano = int('20'+ano[6:])
    return mudaAnoUgestora(ano)
    
    
def manipulaCandidatosCondenadosTRE():
    """
    """
    path_jul = r'~/Downloads/contas_julgadas_irregulares_TRE_2016.csv'
    contas_jul_irr = pd.read_csv(path_jul)
    contas_jul_irr = contas_jul_irr[contas_jul_irr['JURISDICIONADO'].str.contains("Prefeitura")]

    func_padr_nome_resp = np.vectorize(padronizaTexto)
    func_padr_nome_ugest = np.vectorize(nomeUgestora) 
    func_padr_ano = np.vectorize(anoPublPGestao)  
    
    nome_resp = func_padr_nome_resp(contas_jul_irr['RESPONSÁVEL'].values)
    nome_ugest = func_padr_nome_ugest(contas_jul_irr['JURISDICIONADO'].values)
    data_public_gest = func_padr_ano(contas_jul_irr['DATA PUBLICAÇÃO'].values) 
    
    contas_jul_irr.loc[:,('RESPONSÁVEL')] = nome_resp
    contas_jul_irr.loc[:,('JURISDICIONADO')] = nome_ugest
    contas_jul_irr.loc[:,('DATA PUBLICAÇÃO')] = data_public_gest
    
    contas_jul_irr.rename(columns={'JURISDICIONADO':'de_Ugestora', 'DATA PUBLICAÇÃO':'dt_Ano'}, inplace=True)
    return contas_jul_irr

candidatosCondenadosTRE = manipulaCandidatosCondenadosTRE()
candidatosCondenadosTRE.head()

,ITEM,PROCESSO,SUBCATEGORIA,de_Ugestora,CPF,RESPONSÁVEL,DECISÃO,dt_Ano,RES. DECISÃO PODER LEGISLATIVO
0,1,02241/07,PCA - Prestação de Contas Anuais,PUXINANA,109.580.294-15,ABELARDO ANTONIO COUTINHO,PPL-TC 00198/08,2009,"Dec. Leg. 003/2009, rejeitando o Parecer"
1,2,02393/08,PCA - Prestação de Contas Anuais,PUXINANA,109.580.294-15,ABELARDO ANTONIO COUTINHO,PPL-TC 00159/09,2009,"Dec. Leg. 001/2007, rejeitando o Parecer"
2,3,02916/09,PCA - Prestação de Contas Anuais,PUXINANA,109.580.294-15,ABELARDO ANTONIO COUTINHO,PPL-TC 00022/10,2009,"Dec. Leg. 001/2011, rejeitando o Parecer"
3,4,03194/12,PCA - Prestação de Contas Anuais,PUXINANA,109.580.294-15,ABELARDO ANTONIO COUTINHO,APL-TC 00234/13,2013,"Dec. Leg. 002/2013, mantendo o Parecer"
4,5,03194/12,PCA - Prestação de Contas Anuais,PUXINANA,109.580.294-15,ABELARDO ANTONIO COUTINHO,PPL-TC 00051/13,2013,"Dec. Leg. 002/2013, mantendo o Parecer"


## Junção entre as tabelas do TRE e Sagres para gerar as gestões condenáveis.

In [82]:
cand_cond_TREXSagres_Gest = pd.merge(candidatosCondenadosTRE, sagres_pref_eleit_por_ano, on=['de_Ugestora','dt_Ano'])
cand_cond_TREXSagres_Gest.rename(columns={'NOME_CANDIDATO':'ELEITO'}, inplace=True)
cand_cond_TREXSagres_Gest.to_csv('~/Desktop/TRE_Sagres_Resp_Eleito.csv',sep=',',encoding='utf-8', index=False)
cand_cond_TREXSagres_Gest.head()

,ITEM,PROCESSO,SUBCATEGORIA,de_Ugestora,CPF,RESPONSÁVEL,DECISÃO,dt_Ano,RES. DECISÃO PODER LEGISLATIVO,cd_Ugestora,ELEITO
0,10,01910/08,PCA - Prestação de Contas Anuais,SERRA DA RAIZ,409.573.904-59,ADAILMA FERNANDES DA SILVA,PPL-TC 00157/10,2009,NaN,201202,LUIZ GONZAGA BEZERRA DUARTE
1,11,03196/09,PCA - Prestação de Contas Anuais,SERRA DA RAIZ,409.573.904-59,ADAILMA FERNANDES DA SILVA,PPL-TC 00118/10,2009,"Dec. Leg. 004/2011, rejeitando o Parecer",201202,LUIZ GONZAGA BEZERRA DUARTE
2,35,05354/13,PCA - Prestação de Contas Anuais,MALTA,001.214.504-14,AJACIO GOMES WANDERLEY,APL-TC 00833/13,2013,"Dec. Leg. 001/2014, mantendo o Parecer",201109,MANOEL BENEDITO DE LUCENA FILHO
3,36,05354/13,PCA - Prestação de Contas Anuais,MALTA,001.214.504-14,AJACIO GOMES WANDERLEY,PPL-TC 00205/13,2013,"Dec. Leg. 001/2014, mantendo o Parecer",201109,MANOEL BENEDITO DE LUCENA FILHO
4,57,02303/08,PCA - Prestação de Contas Anuais,FREI MARTINHO,NaN,ANA ADELIA NERY CABRAL,APL-TC 00259/12,2009,NaN,201080,FRANCIVALDO SANTOS DE ARAUJO


## Criação da Lista de Prefeitos não condenados.

In [92]:
condicao = sagres_pref_eleit_por_ano['NOME_CANDIDATO'].isin(cand_cond_TREXSagres_Gest['ELEITO'].unique())
cand_idon = sagres_pref_eleit_por_ano[~condicao]
cand_idon.to_csv('~/Desktop/TRE_Sagres_Eleit_Idon.csv',sep=',',encoding='utf-8', index=False)
cand_idon.head()

,cd_Ugestora,dt_Ano,de_Ugestora,NOME_CANDIDATO
0,201002,2009,AGUIAR,MANOEL BATISTA GUEDES FILHO
4,201002,2013,AGUIAR,MANOEL BATISTA GUEDES FILHO
8,201003,2009,ALAGOA GRANDE,JOAO BOSCO CARNEIRO JUNIOR
12,201003,2013,ALAGOA GRANDE,HILDON REGIS NAVARRO FILHO
32,201006,2009,ALCANTIL,JOSE MILTON RODRIGUES


In [90]:
"""
Valida que a intersecao entre o conjunto de condenados e os de canditavos idoneos é conjunto vazio.
"""
canditatos_condenados = set(cand_cond_TREXSagres_Gest['ELEITO'].unique())
canditatos_condenados.intersection(set(cand_idon['NOME_CANDIDATO'].unique()))

set()